In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Circuit tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',]
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6)
})

output_path = rd.rootdir/'output'/'fig_designs'
cache_path = rd.rootdir/'output'/'fig_overview'/'data.gzip'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx'

# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata = base.get_metadata(metadata_path, style='designs')
data = data.merge(metadata, how='left', on='construct')
display(data)

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
designs_palette = metadata_dict['color']
designs_markers = metadata_dict['markers']

In [ ]:
# Gate cells
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
df = data[(data['expressing']) & (data['construct']!='UT') & (data['exp']!='elp_exp61')]

In [ ]:
# Bin data and calculate statistics
df_quantiles, stats, _, fits = base.calculate_bins_stats(df, num_bins=20)
stats = stats.merge(metadata, how='left', on='construct')
fits = fits.merge(metadata, how='left', on='construct')

In [ ]:
ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
metadata['ts_label'] = metadata['ts_kind'].replace(ts_label)

Load PiggyBac data (`data_pb`)

In [ ]:
pb_path = base_path/'2023.07.18_exp63.3-RC'/'export'

plates = pd.DataFrame({
    'data_path': [pb_path],
    
    'yaml_path': [pb_path/'exp63.3_wells2.yaml'],
    
    'biorep': [1],
    
    'exp': ['exp63.3-RC']
})

cache_path = rd.rootdir/'output'/'fig_designs'/'data_pb.gzip'

# Load data
data_pb = pd.DataFrame()
if cache_path.is_file(): data_pb = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data_pb = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_pb = data_pb[data_pb[c]>0]
    
    data_pb.dropna(inplace=True)
    data_pb.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
data_pb = data_pb.merge(metadata, how='left', on='construct')
display(data_pb)

In [ ]:
# Gate cells
# Add missing gates (use gates from tuning exp)
channel_list = ['mGL-A', 'mRuby2-A']
gates.loc[len(gates.index)] = ['exp63.3-RC'] + list(gates.loc[gates['exp']!='exp63.3-RC', channel_list].mean().values) + ['mGL-A','mRuby2-A']  

# Gate data by transfection marker expression
data_pb = data_pb.groupby('exp')[data_pb.columns].apply(lambda x: base.gate_data(x,gates))
data_pb.reset_index(inplace=True, drop=True)
df_pb = data_pb[(data_pb['expressing']) & (data_pb['construct']!='UT')]

In [ ]:
# Bin data and calculate statistics
df_quantiles_pb, stats_pb, _, fits_pb = base.calculate_bins_stats(df_pb, num_bins=20)
stats_pb = stats_pb.merge(metadata, how='left', on='construct')
fits_pb = fits_pb.merge(metadata, how='left', on='construct')

Load data from 293T, MEF lenti infections (`data_lenti`)

In [ ]:
base_path_1 = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.04.05_exp89'/'export'
base_path_2 = rd.datadir/'instruments'/'data'/'attune'/'chris'/'2024.06.02-exp95-lenti-miR-iFFL'/'export'
plate_list = ['_'.join(x) for x in zip(
        ['plate'+str(i) for i in range(1,10)], 
        (['293T']*3 + ['MEF2A']*3 + ['MEF8A']*3),
        ['P9','P14','P15']*3
    )]

plates = pd.DataFrame({
    'data_path': [base_path_1/'293T_control', 
                  base_path_1/'293T_plate1', base_path_1/'293T_plate2', base_path_1/'293T_plate3',
                  base_path_1/'MEF_3_plate1', 
                  base_path_1/'MEF_4-1_plate1', base_path_1/'MEF_4-1_plate2', base_path_1/'MEF_4-1_plate3'] +
                 [base_path_2/p for p in plate_list],
    'yaml_path': [base_path_1/'kasey_yaml2'/'plate_control.yaml', 
                  base_path_1/'kasey_yaml2'/'plate01.yaml', base_path_1/'kasey_yaml2'/'plate02.yaml', base_path_1/'kasey_yaml2'/'plate03.yaml',
                  base_path_1/'kasey_yaml2'/'mef_3_plate01.yaml', 
                  base_path_1/'kasey_yaml2'/'mef_4-1_plate01.yaml', base_path_1/'kasey_yaml2'/'mef_4-1_plate02.yaml', base_path_1/'kasey_yaml2'/'mef_4-1_plate03.yaml'] +
                 [base_path_2/(p+'_metadata.yaml') for p in plate_list],
})

cache_path = output_path/'data_lenti.gzip'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx'

# Load data
data_lenti = pd.DataFrame()
if cache_path.is_file(): data_lenti = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data_lenti = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_lenti = data_lenti[data_lenti[c]>0]
    
    #data.dropna(inplace=True)
    data_lenti.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
data_lenti = data_lenti.merge(metadata, how='left', on='construct')
data_lenti['cell'] = data_lenti['cell_type'].apply(lambda x: x.split('-')[0])
data_lenti['exp'] = data_lenti['cell_type'] + '_' + data_lenti['virus_batch']

def map_biorep(df):
    biorep_map = {val:i for i,val in enumerate(df['exp'].unique())}
    d = df.copy()
    d['biorep'] = d['exp'].map(biorep_map)
    return d

data_lenti = data_lenti.groupby('cell')[data_lenti.columns].apply(map_biorep).reset_index(drop=True)
display(data_lenti)

In [ ]:
# Gate cells
gates_lenti = pd.DataFrame()
channel_list = ['mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates_lenti[channel] = data_lenti[(data_lenti['virus_dilution']==0)].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates_lenti.reset_index(inplace=True)

# Add missing gates
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P10'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P14_'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P16'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values) 
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-3_P10'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P10'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P14'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P16'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)

# Indicate which channels are relevant for each experiment
gates_lenti.sort_values(['exp'], inplace=True)
gates_lenti['marker'] = 'mGL-A'
gates_lenti['output'] = 'mRuby2-A'

# Gate data by marker expression
data_lenti = data_lenti.groupby(['cell_type','virus_batch'])[data_lenti.columns].apply(lambda x: base.gate_data(x,gates_lenti))
data_lenti.reset_index(inplace=True, drop=True)
df_lenti = data_lenti[(data_lenti['expressing']) & (data_lenti['virus_dilution']!=0)]

In [ ]:
# Bin data and calculate statistics
df_lenti['output'] = df_lenti['output'].astype(float)
df_lenti['marker'] = df_lenti['marker'].astype(float)
df_quantiles_lenti, stats_lenti, _, fits_lenti = base.calculate_bins_stats(df_lenti, by=['exp','cell','biorep','construct','dox','virus_dilution'], num_bins=20)
stats_lenti = stats_lenti.merge(metadata, how='left', on='construct')
fits_lenti = fits_lenti.merge(metadata, how='left', on='construct')

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(4.48819,1.25+1.5*3)) # 3.34646
fig_gridspec = matplotlib.gridspec.GridSpec(4, 2, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=([1.25]+[1.5]*3), width_ratios=(1,0.75))
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:]),
    'B': fig.add_subfigure(fig_gridspec[1,0]),
    'C': fig.add_subfigure(fig_gridspec[2,0]),
    'D': fig.add_subfigure(fig_gridspec[3,0]),
    'E': fig.add_subfigure(fig_gridspec[1:,1]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))
scatter_kwargs = dict(s=4, jitter=0.1, linewidth=0.5, edgecolor='white')
xlim = (2e3,6e4)
ylim = (1e1,5e3)

fig_path = output_path/'fig_designs.pdf'
fig.savefig(rd.outfile(fig_path))

In [ ]:
def designs_stats_plots(axes, plot_df, plot_df2, marker_baseline=None, ylim=(0,1.2)):

    xlim = (-0.5, len(plot_df['design'].unique())-0.5)

    # stat gmean
    ax = axes[0]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                        legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Output mean', xlim=xlim, xlabel='design', ylabel='', yscale='log',)
    if marker_baseline: ax.axhline(marker_baseline, color='black', ls=':')
    ax.minorticks_off()

    # slope
    ax = axes[1]
    for construct, group in plot_df2.groupby('construct'):
        sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                        legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='', ylim=ylim,
        yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

    for ax in axes: sns.despine(ax=ax)

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.7, bottom=0.35, top=0.35, right=0.15)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.45))

# transfection
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['ts_num']==1)) | (stats['group']=='base')) &
                       (stats['promoter']=='EF1a')]
plot_df2 = fits[(((fits['miR']=='miRE.FF4') & (fits['group']=='controller') & (fits['ts_num']==1)) | (fits['group']=='base')) &
                       (fits['promoter']=='EF1a')]
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()

designs_stats_plots(axes, plot_df, plot_df2, marker_baseline)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.7, bottom=0.35, top=0.35, right=0.15)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.4))

# PiggyBac integration
plot_df = stats_pb
plot_df2 = fits_pb
designs_stats_plots(axes, plot_df, plot_df2)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.7, bottom=0.35, top=0.35, right=0.15)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.4))

# lenti (HEK293T)
plot_df = stats_lenti[stats_lenti['group'].isin(['controller','base']) & (stats_lenti['dox']==1000) & (stats_lenti['virus_dilution']==1) & 
                      (stats_lenti['cell']=='293T') & ~((stats_lenti['biorep']<4) & (stats_lenti['design']==3))]
plot_df2 = fits_lenti[fits_lenti['group'].isin(['controller','base']) & (fits_lenti['dox']==1000) & (fits_lenti['virus_dilution']==1) & 
                      (fits_lenti['cell']=='293T')&  ~((fits_lenti['biorep']<4) & (fits_lenti['design']==3))]
marker_baseline = np.mean(data_lenti[(data_lenti['construct']=='UT') & (data_lenti['cell_type']=='293T')].groupby(['exp','biorep'])['output'].apply(sp.stats.gmean))

designs_stats_plots(axes, plot_df, plot_df2, marker_baseline, ylim=(0,1))
axes[0].minorticks_on()
axes[0].xaxis.set_tick_params(which='minor', bottom=False)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.55, bottom=0.35, top=0.35, right=0.05)
axes = subfig.subplots(3,2,  gridspec_kw=dict(width_ratios=(1,0.4), hspace=0.88))

plot_df = df_quantiles_lenti[(df_quantiles_lenti['group'].isin(['controller','base'])) & (df_quantiles_lenti['dox']==1000) & 
                             (df_quantiles_lenti['virus_dilution']==1) & (df_quantiles_lenti['cell']=='293T') &
                             (((df_quantiles_lenti['biorep']==1) & (df_quantiles_lenti['design'].isin([1,2]))) | 
                              ((df_quantiles_lenti['biorep']==5) & (df_quantiles_lenti['design'].isin([0,3]))))]
designs_list = [[0,1], [2], [3]]
marker_baseline = np.mean(data_lenti[(data_lenti['construct']=='UT') & (data_lenti['cell_type']=='293T')].groupby(['exp','biorep'])['output'].apply(sp.stats.gmean))

for i in range(3):

    # line plot
    ax = axes[i,0]
    d = plot_df[plot_df['design'].isin(designs_list[i])].copy()
    sns.lineplot(data=d, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
                legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax, markersize=4, markeredgewidth=0.5,
                estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
    ax.set(xscale='log', yscale='log', xlabel='marker', ylim=ylim, xlim=xlim)
    sns.despine(ax=ax)
    ax.axhline(marker_baseline, color='black', ls=':')
    ax.minorticks_on()
    ax.yaxis.set_tick_params(which='minor', left=False)

    # histogram
    ax = axes[i,1]
    sns.kdeplot(data=d, y='output', hue='construct', palette=designs_palette, 
                legend=False, log_scale=True, common_norm=False, ax=ax)
    sns.despine(ax=ax, bottom=True)
    ax.set(xlabel='', ylim=axes[i,0].get_ylim(), ylabel='', yticklabels=[])
    ax.get_xaxis().set_visible(False)
    ax.minorticks_off()

axes[0,0].set_title('Design 1', color=base.colors['teal'])
axes[1,0].set_title('Design 2', color=base.colors['orange'])
axes[2,0].set_title('Design 3', color=base.colors['red'])

fig.savefig(rd.outfile(fig_path))